In [1]:
!pip install timm

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/N/soft/rhel7/deeplearning/Python-3.9.8/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
import torch
import torch.nn as nn
import torch.utils.checkpoint as checkpoint
from timm.models.layers import DropPath, to_2tuple, trunc_normal_

# MLP

In [3]:
class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

# Window Operation

In [4]:
x = torch.rand(size=(1,224,224,3))
x.shape

torch.Size([1, 224, 224, 3])

In [5]:
x = x.view(1, 224 // 7, 7, 224 // 7, 7, 3)
x.shape

torch.Size([1, 32, 7, 32, 7, 3])

In [6]:
x = x.permute(0, 1, 3, 2, 4, 5)
x.shape

torch.Size([1, 32, 32, 7, 7, 3])

In [7]:
x = x.contiguous().view(-1, 7, 7, 3)
x.shape

torch.Size([1024, 7, 7, 3])

In [8]:
def window_partition(x, window_size):
    """
    Args:
        x: (B, H, W, C)
        window_size (int): window size
    Returns:
        windows: (num_windows*B, window_size, window_size, C)
    """
    B, H, W, C = x.shape # [B, 224, 224, 3]
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C) # [B, 32, 7, 32, 7, 3]
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C) 
    # [B, 32, 7, 32, 7, 3] -> [B, 32, 32, 7, 7, 3] -> [(B*32*32),7,7,3]
    return windows

In [9]:
x = x.view(1, 224 // 7, 224 // 7, 7, 7, -1)
x.shape

torch.Size([1, 32, 32, 7, 7, 3])

In [10]:
x = x.permute(0, 1, 3, 2, 4, 5)
x.shape

torch.Size([1, 32, 7, 32, 7, 3])

In [11]:
def window_reverse(windows, window_size, H, W):
    """
    Args:
        windows: (num_windows*B, window_size, window_size, C)
        window_size (int): Window size
        H (int): Height of image
        W (int): Width of image
    Returns:
        x: (B, H, W, C)
    """
    B = int(windows.shape[0] / (H * W / window_size / window_size)) 
    x = windows.view(B, H // window_size,  W // window_size, window_size, window_size, -1)
    # [(B*32*32),7,7,3] -> [B, 32, 32, 7, 7, 3]
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)
    # [B, 32, 32, 7, 7, 3] -> [B, 32, 7, 32, 7, 3] -> [B, 224, 224, 3]
    return x

# Relative Position Bias

In [12]:
class WindowAttention(nn.Module):
    r""" Window based multi-head self attention (W-MSA) module with relative position bias.
    It supports both of shifted and non-shifted window.
    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
    """

    def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):

        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wh, Ww
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads))  # 2*Wh-1 * 2*Ww-1, nH
        # [13, 13, 12]

        # get pair-wise relative position index for each token inside the window
        coords_h = torch.arange(self.window_size[0])
        #print('coords_h: ',coords_h) #7
        coords_w = torch.arange(self.window_size[1])
        #print('coords_w: ',coords_w) #7
        coords = torch.stack(torch.meshgrid([coords_h, coords_w]))  # 2, Wh, Ww
        #print('meshgrid: ',torch.meshgrid([coords_h, coords_w]))
        #print('coords: ',coords.shape) #[2,7,7]
        #print('coords: ',coords)
        
        coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
        #print('coords_flatten: ',coords_flatten.shape) [2, 49]
        #print('coords_flatten: ',coords_flatten)
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
        #print('coords_flatten_1: ',coords_flatten[:, :, None].shape) #[2,49,1]
        #print('coords_flatten_1: ',coords_flatten[:, :, None])
        #print('coords_flatten_2: ',coords_flatten[:, None, :].shape) #[2,1,49]
        #print('coords_flatten_2: ',coords_flatten[:, None, :])
        #print('relative_coords_1: ',relative_coords.shape) #[2,49,49]
        #print('relative_coords_1: ',relative_coords)
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
        #print('relative_coords_2: ',relative_coords.shape) # [49,49,2] 
        #print('relative_coords_2: ',relative_coords)
        relative_coords[:, :, 0] += self.window_size[0] - 1  # shift to start from 0
        #print('relative_coords_3: ',relative_coords) # add 6 to row
        relative_coords[:, :, 1] += self.window_size[1] - 1
        #print('relative_coords_4: ',relative_coords) # add 6 to column
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        #print('relative_coords_5: ',relative_coords) # multiply 13 to rwo 
        relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
        #print('relative_index: ',relative_position_index.shape) [49,49]
        #print('relative_index: ',relative_position_index)
        self.register_buffer("relative_position_index", relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        """
        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None
        """
        B_, N, C = x.shape #[B_=(B*32*32), 49, 96]
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        #print('qkv: ',qkv.shape)
        #[B*1024, 49, 96*3] -> [B*1024,49,3,12,8] -> [3,B*1024,12,49,8]
        q, k, v = qkv[0], qkv[1], qkv[2]  # make torchscript happy (cannot use tensor as tuple)
        #[B*1024, 12, 49, 8]

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))
        #[B*1024, 12, 49, 8] * [B*1024, 12, 8, 49] -> [B*1024, 12, 49, 49]
        #print('attn_0: ',attn.shape)

        relative_position_bias = self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)  
        # Wh*Ww,Wh*Ww,nH
        #print('self.relative_position_index_0: ',self.relative_position_index.shape) #[49, 49]
        #print('self.relative_position_index_1: ',self.relative_position_index.view(-1).shape) #[49*49=2401]
        #print('self.relative_position_bias_table_0: ', \
              #self.relative_position_bias_table[self.relative_position_index.view(-1)].shape)#[2401, 12]
        #print('relative_position_bias_0: ',relative_position_bias.shape) #[49, 49, 12]
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww
        #print('relative_position_bias_1: ',relative_position_bias.shape) #[12, 49, 49] 
        attn = attn + relative_position_bias.unsqueeze(0)
        #print('attn_1: ',attn.shape) [B*1024, 12, 49, 49]

        if mask is not None:
            nW = mask.shape[0] # mask [1024, 49, 49] -> [1024,1,49,49] -> [1,1024,1,49,49]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            #[B*1024, 12, 49, 49] -> [B,1024,12,49,49]
            attn = attn.view(-1, self.num_heads, N, N) # [B,1024,12,49,49]->[B*1024,12,49,49]
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)
        #print('attn_2: ',attn.shape) #[B*1024, 12, 49, 49]

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        #print('attn @ v 1: ',(attn @ v).shape) #[B*1024, 12, 49, 8]
        #print('attn @ v 2: ',(attn @ v).transpose(1, 2).shape) #[B*1024, 49, 12, 8]
        #print('x: ',x.shape) #[B*1024, 49, 96]
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

    def extra_repr(self) -> str:
        return f'dim={self.dim}, window_size={self.window_size}, num_heads={self.num_heads}'

    def flops(self, N):
        # calculate flops for 1 window with token length of N
        flops = 0
        # qkv = self.qkv(x)
        flops += N * self.dim * 3 * self.dim
        # attn = (q @ k.transpose(-2, -1))
        flops += self.num_heads * N * (self.dim // self.num_heads) * N
        #  x = (attn @ v)
        flops += self.num_heads * N * N * (self.dim // self.num_heads)
        # x = self.proj(x)
        flops += N * self.dim * self.dim
        return flops

In [13]:
o = WindowAttention(dim=96, window_size=(7,7), num_heads=12, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.)

/N/soft/rhel7/deeplearning/Python-3.9.8/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [14]:
t0 = torch.rand(size=(1024,49,96))

In [15]:
o(t0)

tensor([[[ 0.1074,  0.3787, -0.0495,  ..., -0.1143,  0.0556, -0.1832],
         [ 0.1077,  0.3795, -0.0492,  ..., -0.1151,  0.0564, -0.1845],
         [ 0.1077,  0.3785, -0.0491,  ..., -0.1154,  0.0560, -0.1842],
         ...,
         [ 0.1078,  0.3795, -0.0494,  ..., -0.1147,  0.0559, -0.1839],
         [ 0.1065,  0.3793, -0.0491,  ..., -0.1146,  0.0563, -0.1842],
         [ 0.1069,  0.3795, -0.0492,  ..., -0.1144,  0.0557, -0.1845]],

        [[ 0.1038,  0.3915, -0.0428,  ..., -0.1079,  0.0811, -0.1960],
         [ 0.1037,  0.3900, -0.0431,  ..., -0.1085,  0.0816, -0.1974],
         [ 0.1034,  0.3910, -0.0430,  ..., -0.1085,  0.0806, -0.1968],
         ...,
         [ 0.1037,  0.3898, -0.0429,  ..., -0.1079,  0.0811, -0.1961],
         [ 0.1026,  0.3912, -0.0433,  ..., -0.1070,  0.0824, -0.1954],
         [ 0.1048,  0.3908, -0.0430,  ..., -0.1082,  0.0806, -0.1965]],

        [[ 0.1098,  0.4013, -0.0395,  ..., -0.1147,  0.0772, -0.1957],
         [ 0.1107,  0.4000, -0.0392,  ..., -0

In [16]:
class WindowAttention(nn.Module):
    r""" Window based multi-head self attention (W-MSA) module with relative position bias.
    It supports both of shifted and non-shifted window.
    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
    """

    def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):

        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wh, Ww
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads))  # 2*Wh-1 * 2*Ww-1, nH

        # get pair-wise relative position index for each token inside the window
        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        coords = torch.stack(torch.meshgrid([coords_h, coords_w]))  # 2, Wh, Ww
        coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
        relative_coords[:, :, 0] += self.window_size[0] - 1  # shift to start from 0
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
        self.register_buffer("relative_position_index", relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        """
        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None
        """
        B_, N, C = x.shape
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # make torchscript happy (cannot use tensor as tuple)

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))

        relative_position_bias = self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)  # Wh*Ww,Wh*Ww,nH
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww
        attn = attn + relative_position_bias.unsqueeze(0)

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

    def extra_repr(self) -> str:
        return f'dim={self.dim}, window_size={self.window_size}, num_heads={self.num_heads}'

    def flops(self, N):
        # calculate flops for 1 window with token length of N
        flops = 0
        # qkv = self.qkv(x)
        flops += N * self.dim * 3 * self.dim
        # attn = (q @ k.transpose(-2, -1))
        flops += self.num_heads * N * (self.dim // self.num_heads) * N
        #  x = (attn @ v)
        flops += self.num_heads * N * N * (self.dim // self.num_heads)
        # x = self.proj(x)
        flops += N * self.dim * self.dim
        return flops

# Shifted Window

In [17]:
py_list = ['P', 'y', 't', 'h', 'o', 'n']
py_tuple = ('P', 'y', 't', 'h', 'o', 'n')

# contains indices -1, -2 and -3
slice_object = slice(0, -4) 

print(py_list[slice_object])  # ['n', 'o', 'h']

# contains indices -1 and -3
slice_object = slice(-2,None)

print(py_tuple[slice_object]) # ('n', 'h')

['P', 'y']
('o', 'n')


In [18]:
img_mask = torch.zeros((1, 8, 8, 1))
h_slices = (slice(0, -4),slice(-4, -2),slice(-2, None))
w_slices = (slice(0, -4),slice(-4, -2),slice(-2, None))
print('img_mask: ',img_mask)

cnt = 0
for h in h_slices:
    for w in w_slices:
        print('h_slices: ',h)
        print('w_slices: ',w)
        print('cnt: ',cnt)
        img_mask[:, h, w, :] = cnt
                    #torch.set_printoptions(profile="full")
        print('img_mask: ',img_mask)
        cnt += 1
            

img_mask:  tensor([[[[0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.]],

         [[0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.]],

         [[0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.]],

         [[0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.]],

         [[0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.]],

         [[0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.]],

         [[0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.]],

         [[0.],
          [0.],
          [0.],
          [0.],
          [0.],

In [19]:
test0 = torch.tensor([[0,0],
                     [1,2]])
print(test0)
test0.unsqueeze(1)-test0.unsqueeze(2)

tensor([[0, 0],
        [1, 2]])


tensor([[[ 0,  0],
         [ 0,  0]],

        [[ 0,  1],
         [-1,  0]]])

In [20]:
mask_windows = window_partition(img_mask, 4)  # nW, window_size, window_size, 1
print('mask_windows_0: ',mask_windows)
print('mask_windows_0: ',mask_windows.shape) #[1,8,8,1]->[4, 4, 4, 1]
mask_windows = mask_windows.view(-1, 16)
print('mask_windows_1: ',mask_windows)
print('mask_windows_1: ',mask_windows.shape) #[4,16]
attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
print('mask_windows.unsqueeze(1): ',mask_windows.unsqueeze(1)) #[4,1,16]
print('mask_windows.unsqueeze(2): ',mask_windows.unsqueeze(2)) #[4,16,1]
torch.set_printoptions(profile="full")
print('attn_mask_0: ',attn_mask)
print('attn_mask_0: ',attn_mask.shape) #[4, 16, 16]
attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
print('attn_mask_1: ',attn_mask)
torch.set_printoptions(profile="default")

mask_windows_0:  tensor([[[[0.],
          [0.],
          [0.],
          [0.]],

         [[0.],
          [0.],
          [0.],
          [0.]],

         [[0.],
          [0.],
          [0.],
          [0.]],

         [[0.],
          [0.],
          [0.],
          [0.]]],


        [[[1.],
          [1.],
          [2.],
          [2.]],

         [[1.],
          [1.],
          [2.],
          [2.]],

         [[1.],
          [1.],
          [2.],
          [2.]],

         [[1.],
          [1.],
          [2.],
          [2.]]],


        [[[3.],
          [3.],
          [3.],
          [3.]],

         [[3.],
          [3.],
          [3.],
          [3.]],

         [[6.],
          [6.],
          [6.],
          [6.]],

         [[6.],
          [6.],
          [6.],
          [6.]]],


        [[[4.],
          [4.],
          [5.],
          [5.]],

         [[4.],
          [4.],
          [5.],
          [5.]],

         [[7.],
          [7.],
          [8.],
     

In [21]:
#o1 = SwinTransformerBlock(dim=96, input_resolution=(224,224), num_heads=12, window_size=7, shift_size=2)

In [22]:
t1 = torch.arange(25*3).view(1, 5, 5, 3)
print(t1)
shifted_t1 = torch.roll(t1, shifts=(-2, -2), dims=(1, 2))
print(shifted_t1)



tensor([[[[ 0,  1,  2],
          [ 3,  4,  5],
          [ 6,  7,  8],
          [ 9, 10, 11],
          [12, 13, 14]],

         [[15, 16, 17],
          [18, 19, 20],
          [21, 22, 23],
          [24, 25, 26],
          [27, 28, 29]],

         [[30, 31, 32],
          [33, 34, 35],
          [36, 37, 38],
          [39, 40, 41],
          [42, 43, 44]],

         [[45, 46, 47],
          [48, 49, 50],
          [51, 52, 53],
          [54, 55, 56],
          [57, 58, 59]],

         [[60, 61, 62],
          [63, 64, 65],
          [66, 67, 68],
          [69, 70, 71],
          [72, 73, 74]]]])
tensor([[[[36, 37, 38],
          [39, 40, 41],
          [42, 43, 44],
          [30, 31, 32],
          [33, 34, 35]],

         [[51, 52, 53],
          [54, 55, 56],
          [57, 58, 59],
          [45, 46, 47],
          [48, 49, 50]],

         [[66, 67, 68],
          [69, 70, 71],
          [72, 73, 74],
          [60, 61, 62],
          [63, 64, 65]],

         [[ 6,  7,  8],

In [23]:
class SwinTransformerBlock(nn.Module):
    r""" Swin Transformer Block.
    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resolution.
        num_heads (int): Number of attention heads.
        window_size (int): Window size.
        shift_size (int): Shift size for SW-MSA.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set.
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float, optional): Stochastic depth rate. Default: 0.0
        act_layer (nn.Module, optional): Activation layer. Default: nn.GELU
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self, dim, input_resolution, num_heads, window_size=7, shift_size=0,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        if min(self.input_resolution) <= self.window_size:
            # if window size is larger than input resolution, we don't partition windows
            self.shift_size = 0
            self.window_size = min(self.input_resolution)
        assert 0 <= self.shift_size < self.window_size, "shift_size must in 0-window_size"
        
        #print('self.window_size: ',self.window_size) #7
        #print('self.shift_size: ',self.shift_size) #2

        self.norm1 = norm_layer(dim)
        self.attn = WindowAttention(
            dim, window_size=to_2tuple(self.window_size), num_heads=num_heads,
            qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
        #print('self.attn: ',self.attn)

        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        #print('mlp_hidden_dim: ',mlp_hidden_dim) # 96*4=284
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        if self.shift_size > 0:
            # calculate attention mask for SW-MSA
            H, W = self.input_resolution
            img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
            #print('img_mask: ',img_mask.shape) [1, 224, 224, 1]
            h_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            #print('slice(0, -self.window_size): ',slice(0, -self.window_size))
            #print('slice(-self.window_size, -self.shift_size): ',slice(-self.window_size, -self.shift_size))
            #print('slice(-self.shift_size, None): ',slice(-self.shift_size, None))
            #print('h_slices: ',h_slices)
           
            w_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            #print('w_slices: ',w_slices)
            
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    #torch.set_printoptions(profile="full")
                    #print('img_mask: ',img_mask) #[1, 224, 224, 1]
                    cnt += 1
                    #print('cnt: ',cnt)

            mask_windows = window_partition(img_mask, self.window_size)  # nW, window_size, window_size, 1
            #print('mask_windows_0: ',mask_windows)
            #print('mask_windows_0: ',mask_windows.shape) #[1024, 7, 7, 1]
            mask_windows = mask_windows.view(-1, self.window_size * self.window_size)
            #print('mask_windows_1: ',mask_windows)
            #print('mask_windows_1: ',mask_windows.shape) #[1024, 49]
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            #print('mask_windows.unsqueeze(1): ',mask_windows.unsqueeze(1).shape) #[1024,1,49]
            #print('mask_windows.unsqueeze(2): ',mask_windows.unsqueeze(2).shape) #[1024,49,1]
            #print('attn_mask_0: ',attn_mask)
            #print('attn_mask_0: ',attn_mask.shape) #[1024, 49, 49]
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
            #print('attn_mask_1: ',attn_mask)
            #print('attn_mask_1: ',attn_mask.shape)
        else:
            attn_mask = None

        self.register_buffer("attn_mask", attn_mask)

    def forward(self, x):
        H, W = self.input_resolution
        B, L, C = x.shape #[B, 224*224, C]
        assert L == H * W, "input feature has wrong size"

        shortcut = x #[B, 224*224, C]
        x = self.norm1(x)
        x = x.view(B, H, W, C) #[B, 224, 224, C]

        # cyclic shift
        if self.shift_size > 0:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
            #[B, 224, 224, C]
        else:
            shifted_x = x

        # partition windows
        x_windows = window_partition(shifted_x, self.window_size)  # nW*B, window_size, window_size, C
        #[B, 224, 224, C] -> [(B*32*32), 7, 7, C]
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)  # nW*B, window_size*window_size, C
        #[(B*32*32),7,7,C] -> [(B*32*32),49,C]
        
        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows, mask=self.attn_mask)  # nW*B, window_size*window_size, C
        #[(B*1024),49,C]

        # merge windows
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)
        #[(B*1024),49,C] -> [(B*1024),7,7,C]
        shifted_x = window_reverse(attn_windows, self.window_size, H, W)  # B H' W' C
        #[(B*1024),7,7,C] -> [B, 224, 224, C]

        # reverse cyclic shift
        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x
        x = x.view(B, H * W, C) #[B, 224*224, C]

        # FFN
        x = shortcut + self.drop_path(x)
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x

    def extra_repr(self) -> str:
        return f"dim={self.dim}, input_resolution={self.input_resolution}, num_heads={self.num_heads}, " \
               f"window_size={self.window_size}, shift_size={self.shift_size}, mlp_ratio={self.mlp_ratio}"

    def flops(self):
        flops = 0
        H, W = self.input_resolution
        # norm1
        flops += self.dim * H * W
        # W-MSA/SW-MSA
        nW = H * W / self.window_size / self.window_size
        flops += nW * self.attn.flops(self.window_size * self.window_size)
        # mlp
        flops += 2 * H * W * self.dim * self.dim * self.mlp_ratio
        # norm2
        flops += self.dim * H * W
        return flops

In [24]:
class SwinTransformerBlock(nn.Module):
    r""" Swin Transformer Block.
    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resulotion.
        num_heads (int): Number of attention heads.
        window_size (int): Window size.
        shift_size (int): Shift size for SW-MSA.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set.
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float, optional): Stochastic depth rate. Default: 0.0
        act_layer (nn.Module, optional): Activation layer. Default: nn.GELU
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self, dim, input_resolution, num_heads, window_size=7, shift_size=0,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        if min(self.input_resolution) <= self.window_size:
            # if window size is larger than input resolution, we don't partition windows
            self.shift_size = 0
            self.window_size = min(self.input_resolution)
        assert 0 <= self.shift_size < self.window_size, "shift_size must in 0-window_size"

        self.norm1 = norm_layer(dim)
        self.attn = WindowAttention(
            dim, window_size=to_2tuple(self.window_size), num_heads=num_heads,
            qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)

        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        if self.shift_size > 0:
            # calculate attention mask for SW-MSA
            H, W = self.input_resolution
            img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
            h_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            w_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    cnt += 1

            mask_windows = window_partition(img_mask, self.window_size)  # nW, window_size, window_size, 1
            mask_windows = mask_windows.view(-1, self.window_size * self.window_size)
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        else:
            attn_mask = None

        self.register_buffer("attn_mask", attn_mask)

    def forward(self, x):
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"

        shortcut = x
        x = self.norm1(x)
        x = x.view(B, H, W, C)

        # cyclic shift
        if self.shift_size > 0:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
        else:
            shifted_x = x

        # partition windows
        x_windows = window_partition(shifted_x, self.window_size)  # nW*B, window_size, window_size, C
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)  # nW*B, window_size*window_size, C

        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows, mask=self.attn_mask)  # nW*B, window_size*window_size, C

        # merge windows
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)
        shifted_x = window_reverse(attn_windows, self.window_size, H, W)  # B H' W' C

        # reverse cyclic shift
        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x
        x = x.view(B, H * W, C)

        # FFN
        x = shortcut + self.drop_path(x)
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x

    def extra_repr(self) -> str:
        return f"dim={self.dim}, input_resolution={self.input_resolution}, num_heads={self.num_heads}, " \
               f"window_size={self.window_size}, shift_size={self.shift_size}, mlp_ratio={self.mlp_ratio}"

    def flops(self):
        flops = 0
        H, W = self.input_resolution
        # norm1
        flops += self.dim * H * W
        # W-MSA/SW-MSA
        nW = H * W / self.window_size / self.window_size
        flops += nW * self.attn.flops(self.window_size * self.window_size)
        # mlp
        flops += 2 * H * W * self.dim * self.dim * self.mlp_ratio
        # norm2
        flops += self.dim * H * W
        return flops

# Patch Merging

In [25]:
class PatchMerging(nn.Module):
    r""" Patch Merging Layer.
    Args:
        input_resolution (tuple[int]): Resolution of input feature.
        dim (int): Number of input channels.
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self, input_resolution, dim, norm_layer=nn.LayerNorm):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.reduction = nn.Linear(4 * dim, 2 * dim, bias=False)
        self.norm = norm_layer(4 * dim)

    def forward(self, x):
        """
        x: B, H*W, C
        """
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"
        assert H % 2 == 0 and W % 2 == 0, f"x size ({H}*{W}) are not even."

        x = x.view(B, H, W, C)

        x0 = x[:, 0::2, 0::2, :]  # B H/2 W/2 C
        x1 = x[:, 1::2, 0::2, :]  # B H/2 W/2 C
        x2 = x[:, 0::2, 1::2, :]  # B H/2 W/2 C
        x3 = x[:, 1::2, 1::2, :]  # B H/2 W/2 C
        x = torch.cat([x0, x1, x2, x3], -1)  # B H/2 W/2 4*C
        x = x.view(B, -1, 4 * C)  # B H/2*W/2 4*C

        x = self.norm(x)
        x = self.reduction(x) # B H/2*W/2 2C

        return x

    def extra_repr(self) -> str:
        return f"input_resolution={self.input_resolution}, dim={self.dim}"

    def flops(self):
        H, W = self.input_resolution
        flops = H * W * self.dim
        flops += (H // 2) * (W // 2) * 4 * self.dim * 2 * self.dim
        return flops

In [26]:
class BasicLayer(nn.Module):
    """ A basic Swin Transformer layer for one stage.
    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resolution.
        depth (int): Number of blocks.
        num_heads (int): Number of attention heads.
        window_size (int): Local window size.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set.
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float | tuple[float], optional): Stochastic depth rate. Default: 0.0
        norm_layer (nn.Module, optional): Normalization layer. Default: nn.LayerNorm
        downsample (nn.Module | None, optional): Downsample layer at the end of the layer. Default: None
        use_checkpoint (bool): Whether to use checkpointing to save memory. Default: False.
    """

    def __init__(self, dim, input_resolution, depth, num_heads, window_size,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., norm_layer=nn.LayerNorm, downsample=None, use_checkpoint=False):

        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.depth = depth
        self.use_checkpoint = use_checkpoint

        # build blocks
        self.blocks = nn.ModuleList([
            SwinTransformerBlock(dim=dim, input_resolution=input_resolution,
                                 num_heads=num_heads, window_size=window_size,
                                 shift_size=0 if (i % 2 == 0) else window_size // 2,
                                 mlp_ratio=mlp_ratio,
                                 qkv_bias=qkv_bias, qk_scale=qk_scale,
                                 drop=drop, attn_drop=attn_drop,
                                 drop_path=drop_path[i] if isinstance(drop_path, list) else drop_path,
                                 norm_layer=norm_layer)
            for i in range(depth)])

        # patch merging layer
        if downsample is not None:
            self.downsample = downsample(input_resolution, dim=dim, norm_layer=norm_layer)
        else:
            self.downsample = None

    def forward(self, x):
        for blk in self.blocks:
            print('basic-layer: ')
            print(x.shape)
            if self.use_checkpoint:
                x = checkpoint.checkpoint(blk, x)
            else:
                x = blk(x)
        if self.downsample is not None:
            
            x = self.downsample(x)
            print('basic-layer-downsample: ')
            print(x.shape)
        return x

    def extra_repr(self) -> str:
        return f"dim={self.dim}, input_resolution={self.input_resolution}, depth={self.depth}"

    def flops(self):
        flops = 0
        for blk in self.blocks:
            flops += blk.flops()
        if self.downsample is not None:
            flops += self.downsample.flops()
        return flops

In [27]:
class PatchEmbed(nn.Module):
    r""" Image to Patch Embedding
    Args:
        img_size (int): Image size.  Default: 224.
        patch_size (int): Patch token size. Default: 4.
        in_chans (int): Number of input image channels. Default: 3.
        embed_dim (int): Number of linear projection output channels. Default: 96.
        norm_layer (nn.Module, optional): Normalization layer. Default: None
    """

    def __init__(self, img_size=224, patch_size=4, in_chans=3, embed_dim=96, norm_layer=None):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        patches_resolution = [img_size[0] // patch_size[0], img_size[1] // patch_size[1]]
        self.img_size = img_size
        self.patch_size = patch_size
        self.patches_resolution = patches_resolution #[56, 56]
        self.num_patches = patches_resolution[0] * patches_resolution[1] #[56*56]

        self.in_chans = in_chans
        self.embed_dim = embed_dim

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
        if norm_layer is not None:
            self.norm = norm_layer(embed_dim)
        else:
            self.norm = None

    def forward(self, x):
        B, C, H, W = x.shape
        print('patch-emd-0: ',x.shape)
        # FIXME look at relaxing size constraints
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        print('patch-emd-1: ',self.proj(x).shape)
        print('patch-emd-2: ',self.proj(x).flatten(2).shape)
        x = self.proj(x).flatten(2).transpose(1, 2)  # B Ph*Pw C
        print('patch-emd-3: ',x.shape)
        if self.norm is not None:
            x = self.norm(x)
        return x

    def flops(self):
        Ho, Wo = self.patches_resolution
        flops = Ho * Wo * self.embed_dim * self.in_chans * (self.patch_size[0] * self.patch_size[1])
        if self.norm is not None:
            flops += Ho * Wo * self.embed_dim
        return flops

In [28]:
class SwinTransformer(nn.Module):
    r""" Swin Transformer
        A PyTorch impl of : `Swin Transformer: Hierarchical Vision Transformer using Shifted Windows`  -
          https://arxiv.org/pdf/2103.14030
    Args:
        img_size (int | tuple(int)): Input image size. Default 224
        patch_size (int | tuple(int)): Patch size. Default: 4
        in_chans (int): Number of input image channels. Default: 3
        num_classes (int): Number of classes for classification head. Default: 1000
        embed_dim (int): Patch embedding dimension. Default: 96
        depths (tuple(int)): Depth of each Swin Transformer layer.
        num_heads (tuple(int)): Number of attention heads in different layers.
        window_size (int): Window size. Default: 7
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim. Default: 4
        qkv_bias (bool): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float): Override default qk scale of head_dim ** -0.5 if set. Default: None
        drop_rate (float): Dropout rate. Default: 0
        attn_drop_rate (float): Attention dropout rate. Default: 0
        drop_path_rate (float): Stochastic depth rate. Default: 0.1
        norm_layer (nn.Module): Normalization layer. Default: nn.LayerNorm.
        ape (bool): If True, add absolute position embedding to the patch embedding. Default: False
        patch_norm (bool): If True, add normalization after patch embedding. Default: True
        use_checkpoint (bool): Whether to use checkpointing to save memory. Default: False
    """

    def __init__(self, img_size=224, patch_size=4, in_chans=3, num_classes=1000,
                 embed_dim=96, depths=[2, 2, 6, 2], num_heads=[3, 6, 12, 24],
                 window_size=7, mlp_ratio=4., qkv_bias=True, qk_scale=None,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0.1,
                 norm_layer=nn.LayerNorm, ape=False, patch_norm=True,
                 use_checkpoint=False, **kwargs):
        super().__init__()

        self.num_classes = num_classes
        self.num_layers = len(depths)
        self.embed_dim = embed_dim
        self.ape = ape
        self.patch_norm = patch_norm
        self.num_features = int(embed_dim * 2 ** (self.num_layers - 1))
        self.mlp_ratio = mlp_ratio

        # split image into non-overlapping patches
        self.patch_embed = PatchEmbed(
            img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim,
            norm_layer=norm_layer if self.patch_norm else None)
        num_patches = self.patch_embed.num_patches
        patches_resolution = self.patch_embed.patches_resolution
        self.patches_resolution = patches_resolution

        # absolute position embedding
        if self.ape:
            self.absolute_pos_embed = nn.Parameter(torch.zeros(1, num_patches, embed_dim))
            trunc_normal_(self.absolute_pos_embed, std=.02)

        self.pos_drop = nn.Dropout(p=drop_rate)

        # stochastic depth
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]  # stochastic depth decay rule
        print('SwinTransformer-dpr: ',dpr)

        # build layers
        self.layers = nn.ModuleList()
        for i_layer in range(self.num_layers):
            
            layer = BasicLayer(dim=int(embed_dim * 2 ** i_layer),
                               input_resolution=(patches_resolution[0] // (2 ** i_layer),
                                                 patches_resolution[1] // (2 ** i_layer)),
                               depth=depths[i_layer],
                               num_heads=num_heads[i_layer],
                               window_size=window_size,
                               mlp_ratio=self.mlp_ratio,
                               qkv_bias=qkv_bias, qk_scale=qk_scale,
                               drop=drop_rate, attn_drop=attn_drop_rate,
                               drop_path=dpr[sum(depths[:i_layer]):sum(depths[:i_layer + 1])],
                               norm_layer=norm_layer,
                               downsample=PatchMerging if (i_layer < self.num_layers - 1) else None,
                               use_checkpoint=use_checkpoint)
            self.layers.append(layer)

        self.norm = norm_layer(self.num_features)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.head = nn.Linear(self.num_features, num_classes) if num_classes > 0 else nn.Identity()

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'absolute_pos_embed'}

    @torch.jit.ignore
    def no_weight_decay_keywords(self):
        return {'relative_position_bias_table'}

    def forward_features(self, x):
        print('SwinTransformer-x0: ',x.shape)
        x = self.patch_embed(x)
        print('SwinTransformer-x1-after_emb: ',x.shape)
        if self.ape:
            x = x + self.absolute_pos_embed
        x = self.pos_drop(x)

        for layer in self.layers:
            x = layer(x)
            print('SwinTransformer-x2-after_basic_layer: ',x.shape)

        x = self.norm(x)  # B L C
        x = self.avgpool(x.transpose(1, 2))  # B C 1
        print('SwinTransformer-x3-after_avgpool: ',x.shape)
        
        x = torch.flatten(x, 1)
        print('SwinTransformer-x4-after_flatten: ',x.shape)
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x

    def flops(self):
        flops = 0
        flops += self.patch_embed.flops()
        for i, layer in enumerate(self.layers):
            flops += layer.flops()
        flops += self.num_features * self.patches_resolution[0] * self.patches_resolution[1] // (2 ** self.num_layers)
        flops += self.num_features * self.num_classes
        return flops

In [29]:
from torchsummary import summary

In [30]:
swin = SwinTransformer(img_size=224, patch_size=4, in_chans=3, num_classes=1000,
                 embed_dim=96, depths=[2, 2, 6, 2], num_heads=[3, 6, 12, 24],
                 window_size=7, mlp_ratio=4., qkv_bias=True, qk_scale=None,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0.1,
                 norm_layer=nn.LayerNorm, ape=False, patch_norm=True,
                 use_checkpoint=False)

SwinTransformer-dpr:  [0.0, 0.00909090880304575, 0.0181818176060915, 0.027272727340459824, 0.036363635212183, 0.045454543083906174, 0.054545458406209946, 0.06363636255264282, 0.0727272778749466, 0.08181818574666977, 0.09090909361839294, 0.10000000149011612]


In [31]:
summary(swin, (3, 224, 224), device='cpu')

SwinTransformer-x0:  torch.Size([2, 3, 224, 224])
patch-emd-0:  torch.Size([2, 3, 224, 224])
patch-emd-1:  torch.Size([2, 96, 56, 56])
patch-emd-2:  torch.Size([2, 96, 3136])
patch-emd-3:  torch.Size([2, 3136, 96])
SwinTransformer-x1-after_emb:  torch.Size([2, 3136, 96])
basic-layer: 
torch.Size([2, 3136, 96])
basic-layer: 
torch.Size([2, 3136, 96])
basic-layer-downsample: 
torch.Size([2, 784, 192])
SwinTransformer-x2-after_basic_layer:  torch.Size([2, 784, 192])
basic-layer: 
torch.Size([2, 784, 192])
basic-layer: 
torch.Size([2, 784, 192])
basic-layer-downsample: 
torch.Size([2, 196, 384])
SwinTransformer-x2-after_basic_layer:  torch.Size([2, 196, 384])
basic-layer: 
torch.Size([2, 196, 384])
basic-layer: 
torch.Size([2, 196, 384])
basic-layer: 
torch.Size([2, 196, 384])
basic-layer: 
torch.Size([2, 196, 384])
basic-layer: 
torch.Size([2, 196, 384])
basic-layer: 
torch.Size([2, 196, 384])
basic-layer-downsample: 
torch.Size([2, 49, 768])
SwinTransformer-x2-after_basic_layer:  torch.S